#### Setup

In [1]:
from google.colab import drive
import sys

In [2]:
drive.mount('/content/drive')
sys.path.append('/content/drive/Shareddrives/CMPE260/')

Mounted at /content/drive


In [3]:
%run /content/drive/Shareddrives/CMPE260/rec_implementation.ipynb

cpu


#### Training

In [4]:
ou_noise = OUStrategy(params['embedding_dim'])

In [6]:
train_env = Environment(train_matrix)

hits, dcgs = [], []
hits_all, dcgs_all = [], []
step, best_step = 0, 0
step, best_step, best_step_all = 0, 0, 0

users = np.random.permutation(apt_users)

In [7]:
for u in tqdm.tqdm(users):
    print('user =', u)
    user, memory = train_env.reset(u)

    for t in range(int(train_matrix[u].sum())):
      try:
        # given current state, get action using actor network
        action_emb = policy_network(state_representation(user, memory))
        action_emb = ou_noise.get_action(action_emb[0], t)
        action = policy_network.get_action(user, torch.tensor(train_env.memory[user.detach().cpu().numpy().astype(int), :]), state_representation, action_emb,
            torch.tensor([item for item in train_env.available_books if item not in train_env.viewed_books]).long()
        )
      except Exception as e:
        print(e)

      # update replay buffer
      user, memory, reward, done = train_env.step(action, action_emb, replay_buffer)

      # update weights of action and critic network
      if len(replay_buffer) > params['batch_size']:
          update(params['batch_size'], params['gamma'], params['min_value'], params['max_value'], params['soft_tau'])

      # monitor training process by evaluating on only one user for efficiency
      if step % 100 == 0 and step > 0:
          hit, dcg = evaluation(policy_network, state_representation, train_env.memory, test_loader)

          hits.append(hit)
          dcgs.append(dcg)

          if np.mean(np.array([hit, dcg]) - np.array([hits[best_step], dcgs[best_step]])) >= 0:
              best_step = step // 100
              torch.save(policy_network.state_dict(), params['log_w_ou_dir'] + 'policy_net.pth')
              torch.save(value_network.state_dict(), params['log_w_ou_dir'] + 'value_net.pth')
              torch.save(state_representation.state_dict(), params['log_w_ou_dir'] + 'state_repr.pth')

      # evaluate for best policy after every 1000 steps on the whole test set
      if step % 1000 == 0 and step > 0:
          hit, dcg = evaluation(policy_network, state_representation, train_env.memory, full_loader)

          hits_all.append(hit)
          dcgs_all.append(dcg)

          if np.mean(np.array([hit, dcg]) - np.array([hits_all[best_step_all], dcgs_all[best_step_all]])) >= 0:
              best_step_all = step // 1000
              torch.save(policy_network.state_dict(), params['log_w_ou_dir'] + 'best_policy_net.pth')
              torch.save(value_network.state_dict(), params['log_w_ou_dir'] + 'best_value_net.pth')
              torch.save(state_representation.state_dict(), params['log_w_ou_dir'] + 'best_state_repr.pth')

      step += 1

  0%|          | 0/52 [00:00<?, ?it/s]

user = 213


  2%|▏         | 1/52 [00:08<07:18,  8.59s/it]

user = 11


  4%|▍         | 2/52 [00:16<06:54,  8.29s/it]

...

 96%|█████████▌| 50/52 [38:16<01:00, 30.07s/it]

user = 242


 98%|█████████▊| 51/52 [38:29<00:24, 24.88s/it]

user = 144


100%|██████████| 52/52 [43:12<00:00, 49.86s/it] 


In [8]:
torch.save(policy_network.state_dict(), params['log_w_ou_dir'] + 'policy_net_final.pth')
torch.save(value_network.state_dict(), params['log_w_ou_dir'] + 'value_net_final.pth')
torch.save(state_representation.state_dict(), params['log_w_ou_dir'] + 'state_repr_final.pth')

In [9]:
with open(params['log_w_ou_dir'] + 'memory.pickle', 'wb') as f:
  pickle.dump(train_env.memory, f)

#### Evaluation on test set

In [10]:
with open(params['log_w_ou_dir'] + 'memory.pickle', 'rb') as f:
    memory = pickle.load(f)

In [11]:
state_rep = State_Representation(num_of_users, num_of_books, params['embedding_dim'], params['hidden_dim'])
policy = Actor(params['embedding_dim'], params['hidden_dim'])
state_rep.load_state_dict(torch.load(params['log_w_ou_dir'] + 'best_state_repr.pth'))
policy.load_state_dict(torch.load(params['log_w_ou_dir'] + 'best_policy_net.pth'))

<All keys matched successfully>

In [12]:
hit, dcg = evaluation(policy, state_rep, memory, full_loader)
print('hit rate: ', hit, 'dcg: ', dcg)

hit rate:  0.9684250117536436 dcg:  0.48148500477136663


#### Inference

In [ ]:
random_user = test_data['user'].sample().iloc[0]
random_user

107

In [ ]:
temp_test_data = test_data[test_data.user == random_user]
temp = pd.merge(temp_test_data, filtered_df, on=['user', 'book'])
temp

,user,book,isbn,title,author,user_id,rating,best_seller
0,107,1341,345309014,Second Foundation (Foundation Novels (Paperback)),Isaac Asimov,254206,9.0,False
1,107,2933,425173534,Ssn,Tom Clancy,254206,9.0,True
2,107,3897,441516645,Glory Lane,Alan Dean Foster,254206,10.0,True
3,107,6824,451151224,The Running Man,Stephen King,254206,9.0,True
4,107,7131,374184232,The Laws of Our Fathers,Scott Turow,254206,9.0,False
5,107,10273,395489334,"The Two Towers (The Lord of the Rings, Part 2)",J. R. R. Tolkien,254206,10.0,True
6,107,11569,671653741,SIXTH COLUMN,Robert A. Heinlein,254206,10.0,True
7,107,15090,345288319,The Empire Strikes Back,Donald F. Glut,254206,9.0,False
8,107,15341,345338200,The Caves of Steel,Isaac Asimov,254206,10.0,True
9,107,15358,441218911,Expanded Universe,Robert Heinlein,254206,10.0,False


In [ ]:
recommendations = []

for model, state_representation in zip([policy], [state_rep]):
    env = Environment(test_matrix)
    user, memory = env.reset(random_user)

    # give 2 books that the user has rated in the past
    user, memory, reward, _ = env.step(torch.tensor([6824]))
    user, memory, reward, _ = env.step(torch.tensor([15341]))

    # get next 3 recommendations based on the two books above
    recs = []
    for i in range(3):
        action_emb = model(state_representation(user, memory))
        action = model.get_action(
            user,
            torch.tensor(env.memory[user.detach().cpu().numpy().astype(int), :]),
            state_representation,
            action_emb,
            torch.tensor([item for item in env.available_books if item not in env.viewed_books]).long()
        )
        user, memory, reward, _ = env.step(action)
        recs.append(action)

    recommendations.append(recs)

In [ ]:
recos = [r.item() for r in recommendations[0]]
recos

[23555, 26263, 27268]

In [ ]:
recommended_books = filtered_df[filtered_df.book.isin(recos)][['title', 'author', 'best_seller']].drop_duplicates()
recommended_books

,title,author,best_seller
301919,Reading the Tarot,Leo Louis Martello,True
323999,"Baby Love (Harlequin Temptation, No 352)",Binnie Syril,False
333473,Women on Men,Marlene Rimler,True
